In [0]:
%%sh


PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -U citus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 
SELECT count(*) FROM store.store_app_rank_fact_v1  WHERE (date='2019-11-05') AND (device_code='ios-phone') AND ( granularity = 'hourly' ) AND ( hour = 23 ) AND (free_download is not null);
SELECT count(*) FROM store.store_app_rank_fact_v1  WHERE (date='2019-11-05') AND (device_code='ios-phone') AND ( granularity = 'hourly' ) AND ( hour = 23 ) AND (new_paid_download is not null);
SELECT count(*) FROM store.store_app_rank_fact_v1  WHERE (date='2019-11-05') AND (device_code='ios-phone') AND ( granularity = 'hourly' ) AND ( hour = 23 ) AND (revenue is not null);
SELECT count(*) FROM store.store_app_rank_fact_v1  WHERE (date='2019-11-05') AND (device_code='ios-phone') AND ( granularity = 'hourly' ) AND ( hour = 23 ) AND (paid_download is not null);
SELECT count(*) FROM store.store_app_rank_fact_v1  WHERE (date='2019-11-05') AND (device_code='ios-phone') AND ( granularity = 'hourly' ) AND ( hour = 23 ) AND (new_free_download is not null);

EOF


In [0]:

from aaintdatapipeline.core.conf import Conf
from aaintdatapipeline.core.fs.device import S3Bucket, specified_bucket
from aaintdatapipeline.core.fs.device import unified_bucket
import zlib

bucket_name = "prod_appannie_ios"
s3 = S3Bucket(Conf(bucket_name = bucket_name))
conf = Conf(
        bucket_name=bucket_name,
        bucket_class=S3Bucket
)
bucket = specified_bucket(conf)
raw_data = zlib.decompress(bucket.get("country-ranks/2019-09-16/23/"))

for _data in raw_data.splitlines():
    print _data
    break
    _data_list = _data.split(",")
    _rank_list = _data_list[4].split(" ")
    print _data_list[:3]
    print len(_rank_list)
    for _rank in _rank_list:
        print _rank
        
        
        
def get_app_store_rank_ios_raw_df(date, country_code):
    """
    :return: raw_data_frame
    :rtype: list_of_dic
    raw_data:
    _________________________________________________________________________________
    |    date    |   country_id   |  category_id  |   feed_id   |   rank (app_id)   |
    |------------|----------------|---------------|-------------|-------------------|
    | 2019-04-27 | 143441(bigint) |   6016 (int)  |   0 (int)   | 376510438(bigint) |
    ---------------------------------------------------------------------------------
    unified_data:
    _____________________________________________________________________________________
    |  country_code  |   category_id   |       app_id       | feed_name (free_download) |
    |----------------|-----------------|--------------------|---------------------------|
    |      'US'      | 100026 (bigint) | 376510438 (bigint) |    25 (int) (app_rank)    |
    -------------------------------------------------------------------------------------
    """
    country_id = get_ios_country_id_from_code(country_code)
    path = "country-ranks/{_date}/23/{_country_id}".format(_date=date, _country_id=country_id)
    bucket = rank_bucket("prod_appannie_ios")
    raw_ios_df = []
    for key in bucket.list(path):
        if 'md5' in key or 'SUCCESS' in key:
            continue
        raw_data = zlib.decompress(bucket.get(key))
        for _data in raw_data.splitlines():
            data = _data.split("\t")
            if int(data[3]) in (200, 201, 202):
                continue
            if len(data) == 5:
                data_dic = {
                    'date': data[0],
                    'country_id': data[1],
                    'category_id': data[2],
                    'feed_id': data[3],
                    'rank': data[4].split(" ")
                }
                raw_ios_df.append(data_dic)
            else:
                print("Error: The length of raw data is wrong, length is {}.".format(len(data)))
        return raw_ios_df
    return raw_ios_df



In [0]:


from aaintdatapipeline.core.conf import Conf
from aaintdatapipeline.core.fs.device import S3Bucket, specified_bucket
from aaintdatapipeline.core.fs.device import unified_bucket
import zlib


COUNTRY_MAPPING = {
    'gp': {1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB',
        8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA',
        15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID',
        22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL',
        29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT',
        36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO',
        43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE',
        50: 'KW', 51: 'SA', 52: 'CO', 53: 'KZ', 54: 'PK', 56: 'PE', 61: 'AZ',
        62: 'EC', 64: 'CR', 65: 'LB', 73: 'QA', 74: 'NG', 78: 'LT', 80: 'HR',
        86: 'LV', 95: 'KE', 1000: 'WW'},
    'ios': {143441: 'US', 143442: 'FR', 143443: 'DE', 143444: 'GB', 143445: 'AT',
        143446: 'BE', 143447: 'FI', 143448: 'GR', 143449: 'IE', 143450: 'IT',
        143451: 'LU', 143452: 'NL', 143453: 'PT', 143454: 'ES', 143455: 'CA',
        143456: 'SE', 143457: 'NO', 143458: 'DK', 143459: 'CH', 143460: 'AU',
        143461: 'NZ', 143462: 'JP', 143463: 'HK', 143464: 'SG', 143465: 'CN',
        143466: 'KR', 143467: 'IN', 143468: 'MX', 143469: 'RU', 143470: 'TW',
        143471: 'VN', 143472: 'ZA', 143473: 'MY', 143474: 'PH', 143475: 'TH',
        143476: 'ID', 143477: 'PK', 143478: 'PL', 143479: 'SA', 143480: 'TR',
        143481: 'AE', 143482: 'HU', 143483: 'CL', 143484: 'NP', 143485: 'PA',
        143486: 'LK', 143487: 'RO', 143489: 'CZ', 143491: 'IL', 143492: 'UA',
        143493: 'KW', 143494: 'HR', 143495: 'CR', 143496: 'SK', 143497: 'LB',
        143498: 'QA', 143499: 'SI', 143501: 'CO', 143502: 'VE', 143503: 'BR',
        143504: 'GT', 143505: 'AR', 143506: 'SV', 143507: 'PE', 143508: 'DO',
        143509: 'EC', 143510: 'HN', 143511: 'JM', 143512: 'NI', 143513: 'PY',
        143514: 'UY', 143515: 'MO', 143516: 'EG', 143517: 'KZ', 143518: 'EE',
        143519: 'LV', 143520: 'LT', 143521: 'MT', 143523: 'MD', 143524: 'AM',
        143525: 'BW', 143526: 'BG', 143528: 'JO', 143529: 'KE', 143530: 'MK',
        143531: 'MG', 143532: 'ML', 143533: 'MU', 143534: 'NE', 143535: 'SN',
        143536: 'TN', 143537: 'UG', 143538: 'AI', 143539: 'BS', 143540: 'AG',
        143541: 'BB', 143542: 'BM', 143543: 'VG', 143544: 'KY', 143545: 'DM',
        143546: 'GD', 143547: 'MS', 143548: 'KN', 143549: 'LC', 143550: 'VC',
        143551: 'TT', 143552: 'TC', 143553: 'GY', 143554: 'SR', 143555: 'BZ',
        143556: 'BO', 143557: 'CY', 143558: 'IS', 143559: 'BH', 143560: 'BN',
        143561: 'NG', 143562: 'OM', 143563: 'DZ', 143564: 'AO', 143565: 'BY',
        143566: 'UZ', 143568: 'AZ', 143571: 'YE', 143572: 'TZ', 143573: 'GH',
        143575: 'AL', 143576: 'BJ', 143577: 'BT', 143578: 'BF', 143579: 'KH',
        143580: 'CV', 143581: 'TD', 143582: 'CG', 143583: 'FJ', 143584: 'GM',
        143585: 'GW', 143586: 'KG', 143587: 'LA', 143588: 'LR', 143589: 'MW',
        143590: 'MR', 143591: 'FM', 143592: 'MN', 143593: 'MZ', 143594: 'NA',
        143595: 'PW', 143597: 'PG', 143598: 'ST', 143599: 'SC', 143600: 'SL',
        143601: 'SB', 143602: 'SZ', 143603: 'TJ', 143604: 'TM', 143605: 'ZW',
        0: 'WW'},
}

CATEGORY_MAPPING = {
    "mac-os-mac": {
        39: 200000,
        12001: 200001,
        12002: 200002,
        12003: 200003,
        12004: 200004,
        12005: 200005,
        12006: 200006,
        12007: 200007,
        12008: 200008,
        12010: 200009,
        12011: 200010,
        12012: 200011,
        12013: 200012,
        12014: 200013,
        12015: 200014,
        12016: 200015,
        12017: 200016,
        12018: 200017,
        12019: 200018,
        12020: 200019,
        12021: 200020,
        12022: 200021,
        12201: 200022,
        12202: 200023,
        12203: 200024,
        12204: 200025,
        12205: 200026,
        12206: 200027,
        12207: 200028,
        12208: 200029,
        12209: 200030,
        12210: 200031,
        12211: 200032,
        12212: 200033,
        12213: 200034,
        12214: 200035,
        12215: 200036,
        12216: 200037,
        12217: 200038,
        12218: 200039,
        12219: 200040,
    },
    "tv-os-tv": {
        36: 300000,
        360: 300001,
        6004: 300002,
        6009: 300003,
        6012: 300004,
        6013: 300005,
        6014: 300006,
        6016: 300007,
        6017: 300008
    }

}


class AppStoreRankRawData():
    bucket_name = ""
    bucket_path = ""
    data_split_str = ""
    rank_split_str = ""
    accept_feeds = []

    def get_raw_data_by_date(self, date):
        """
        :return: raw_data_frame
        :rtype: list_of_dic
        raw_data:
        _________________________________________________________________________________
        |    date    |   country_id   |  category_id  |   feed_id   |   rank (app_id)   |
        |------------|----------------|---------------|-------------|-------------------|
        | 2019-04-27 | 143441(bigint) |   6016 (int)  |   0 (int)   | 376510438(bigint) |
        ---------------------------------------------------------------------------------
        unified_data:
        _____________________________________________________________________________________
        |  country_code  |   category_id   |       app_id       | feed_name (free_download) |
        |----------------|-----------------|--------------------|---------------------------|
        |      'US'      | 100026 (bigint) | 376510438 (bigint) |    25 (int) (app_rank)    |
        -------------------------------------------------------------------------------------
        """
        path = "{_bucket_path}/{_date}/23/".format(_date=date, _bucket_path=self.bucket_path)
        bucket = rank_bucket(self.bucket_name)
        raw_df = []
        count = 0
        category=[]
        country=[]
        country_dict = {}
        category_dict = {}
        for key in bucket.list(path):
            if 'md5' in key or 'SUCCESS' in key:
                continue
            raw_data = zlib.decompress(bucket.get(key))
            for _data in raw_data.splitlines():
                data = _data.split(self.data_split_str)
                # remove tv data
                if int(data[3]) not in self.accept_feeds:
                    continue
                if len(data) == 5:
                    data_dic = {
                        'date': data[0],
                        'country': data[1],
                        'category': int(data[2]),
                        'feed': data[3],
                        'rank': [rank for rank in data[4].split(self.rank_split_str) if rank.strip() != '']
                    }
                    count += len(data_dic['rank'])
                    # country.append(data_dic['country'])
                    # category.append(data_dic['category'])
                    country_code = data_dic['country']
                    category_id = CATEGORY_MAPPING['tv-os-tv'][data_dic['category']]
                    if country_code not in country_dict:
                        country_dict[country_code] = {}
                    if category_id not in country_dict[country_code]:
                        country_dict[country_code][category_id] = 0
                    if category_id==300001 and country_code in ["PT","MR"]:
                        pass
                        # print data_dic
                    country_dict[country_code][category_id] += len(data_dic['rank'])
                    # if country_code=="MS":
                    #     print data_dic
                    #     if data_dic["category"] not in category_dict:
                    #         category_dict[data_dic["category"]] = 0
                    #     category_dict[data_dic["category"]] += len(data_dic["rank"])
                    # print count
                    # raw_df.append(data_dic)
                else:
                    print("Error: The length of raw data is wrong, length is {}.".format(len(data)))
        # print country_dict

        return count
        # print len(set(country))
        # print len(set(category))
        # print category_dict
        # return raw_df


class IosPhoneRaw(AppStoreRankRawData):
    bucket_name = "prod_appannie_ios"
    bucket_path = "country-ranks"
    data_split_str = "\t"
    rank_split_str = " "
    accept_feeds = [0, 1, 2]


class IosTabletRaw(AppStoreRankRawData):
    bucket_name = "prod_appannie_ios"
    bucket_path = "country-ranks"
    data_split_str = "\t"
    rank_split_str = " "
    accept_feeds = [101, 100, 102]


class MacOsMacRaw(AppStoreRankRawData):
    bucket_name = "prod_appannie_ios"
    bucket_path = "mac/country-ranks"
    data_split_str = "\t"
    rank_split_str = " "
    accept_feeds = [200, 201, 202]


class TvOsTvRaw(AppStoreRankRawData):
    bucket_name = "prod_appannie_appletv"
    bucket_path = "country-ranks"
    data_split_str = ","
    rank_split_str = " "
    accept_feeds = [0, 1, 2]


def rank_bucket(bucket_str):
    """
    Get Bucket Object from bucket string
    :param bucket_str: string like: s3://xxx.xxx
    :type bucket_str: str
    :return: bucket_object
    :rtype: bucket_object
    """
    conf = Conf(
        bucket_name=bucket_str,
        bucket_class=S3Bucket
    )
    return specified_bucket(conf)


print TvOsTvRaw().get_raw_data_by_date("2019-07-29")

In [0]:


from pyspark.sql import functions as F


APP_STORE_RANK_METRICS = ["free_download", "new_paid_download", "revenue", "paid_download", "new_free_download"]


# df_unified = df.filter("country_code='MS'").groupBy("category_id").agg(F.count("free_download").alias("free_download_count"),F.count("paid_download").alias("paid_download_count"), F.count("revenue").alias("revenue_count")).collect()

# # for item in df_unified:
#     # print "{},{}".format(item.category_id, sum([item.free_download_count, item.paid_download_count, item.revenue_count]))

# raw_result = {200000: 257, 200001: 33, 200002: 13, 200003: 13, 200004: 10, 200005: 9, 200006: 56, 200007: 3, 200008: 7, 200009: 3, 200010: 23, 200011: 4, 200012: 16, 200013: 78, 200014: 7, 200015: 16, 200016: 3, 200017: 4, 200018: 59, 200019: 27, 200020: 3, 200021: 17, 200022: 20, 200023: 19, 200024: 9, 200025: 7, 200026: 8, 200027: 3, 200028: 3, 200029: 3, 200030: 6, 200031: 3, 200032: 3, 200033: 12, 200034: 3, 200035: 6, 200036: 10, 200037: 4, 200038: 7, 200039: 4, 200040: 5}

# for item in df_unified:
#     category_id = int(item.category_id)
#     print "{} {} {}".format(category_id, raw_result[category_id], sum([item.free_download_count, item.paid_download_count, item.revenue_count]) )
    


# continue
def get_unified_count_by_date(date_str):
    device_code = "tv-os-tv"
    df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/app-tech.store.app-rank.v1/fact/date={}/device_code={}/".format(date_str, device_code))
    count = 0
    for metric in APP_STORE_RANK_METRICS:
        # df.show(1)
        count += df.filter("{} is not null".format(metric)).count()
    return count


In [0]:


raw_result = {'BE': {300000: 600, 300001: 38, 300002: 181, 300003: 200, 300004: 322, 300005: 250, 300006: 600, 300007: 576, 300008: 399}, 'BF': {300000: 226, 300001: 32, 300002: 15, 300003: 20, 300004: 3, 300005: 12, 300006: 173, 300007: 53, 300008: 31}, 'BG': {300000: 600, 300001: 45, 300002: 67, 300003: 100, 300004: 126, 300005: 92, 300006: 555, 300007: 346, 300008: 173}, 'BB': {300000: 256, 300001: 26, 300002: 21, 300003: 33, 300004: 19, 300005: 17, 300006: 74, 300007: 91, 300008: 22}, 'BM': {300000: 326, 300001: 46, 300002: 26, 300003: 31, 300004: 11, 300005: 21, 300006: 194, 300007: 94, 300008: 23}, 'BN': {300000: 376, 300001: 43, 300002: 30, 300003: 26, 300004: 14, 300005: 24, 300006: 286, 300007: 135, 300008: 55}, 'BO': {300000: 559, 300001: 43, 300002: 49, 300003: 39, 300004: 31, 300005: 51, 300006: 389, 300007: 232, 300008: 82}, 'BH': {300000: 505, 300001: 31, 300002: 49, 300003: 62, 300004: 50, 300005: 63, 300006: 346, 300007: 271, 300008: 91}, 'BJ': {300000: 270, 300001: 32, 300002: 14, 300003: 19, 300004: 5, 300005: 11, 300006: 123, 300007: 55, 300008: 15}, 'BT': {300000: 32, 300001: 22, 300002: 3, 300003: 5, 300004: 0, 300005: 2, 300006: 4, 300007: 10, 300008: 2}, 'JM': {300000: 263, 300001: 38, 300002: 31, 300003: 38, 300004: 26, 300005: 19, 300006: 199, 300007: 142, 300008: 29}, 'BW': {300000: 298, 300001: 32, 300002: 29, 300003: 37, 300004: 40, 300005: 24, 300006: 167, 300007: 122, 300008: 43}, 'BR': {300000: 600, 300001: 40, 300002: 213, 300003: 233, 300004: 314, 300005: 271, 300006: 600, 300007: 564, 300008: 439}, 'BS': {300000: 258, 300001: 39, 300002: 23, 300003: 21, 300004: 15, 300005: 20, 300006: 203, 300007: 107, 300008: 19}, 'BY': {300000: 600, 300001: 33, 300002: 56, 300003: 70, 300004: 88, 300005: 73, 300006: 478, 300007: 324, 300008: 112}, 'BZ': {300000: 251, 300001: 33, 300002: 17, 300003: 12, 300004: 10, 300005: 21, 300006: 60, 300007: 84, 300008: 15}, 'RU': {300000: 600, 300001: 24, 300002: 211, 300003: 218, 300004: 364, 300005: 282, 300006: 600, 300007: 600, 300008: 490}, 'TM': {300000: 249, 300001: 36, 300002: 11, 300003: 19, 300004: 4, 300005: 8, 300006: 124, 300007: 58, 300008: 12}, 'TJ': {300000: 187, 300001: 27, 300002: 7, 300003: 15, 300004: 3, 300005: 8, 300006: 56, 300007: 46, 300008: 6}, 'RO': {300000: 600, 300001: 42, 300002: 108, 300003: 124, 300004: 199, 300005: 145, 300006: 600, 300007: 444, 300008: 305}, 'GW': {300000: 62, 300001: 20, 300002: 6, 300003: 12, 300004: 0, 300005: 0, 300006: 22, 300007: 10, 300008: 1}, 'GT': {300000: 600, 300001: 47, 300002: 67, 300003: 72, 300004: 77, 300005: 87, 300006: 494, 300007: 300, 300008: 172}, 'GR': {300000: 600, 300001: 39, 300002: 120, 300003: 125, 300004: 162, 300005: 135, 300006: 600, 300007: 407, 300008: 234}, 'JP': {300000: 600, 300001: 40, 300002: 222, 300003: 234, 300004: 307, 300005: 240, 300006: 600, 300007: 559, 300008: 419}, 'GY': {300000: 157, 300001: 25, 300002: 11, 300003: 15, 300004: 0, 300005: 4, 300006: 38, 300007: 41, 300008: 10}, 'GD': {300000: 118, 300001: 29, 300002: 8, 300003: 6, 300004: 1, 300005: 3, 300006: 35, 300007: 32, 300008: 7}, 'GB': {300000: 600, 300001: 35, 300002: 274, 300003: 246, 300004: 417, 300005: 401, 300006: 600, 300007: 600, 300008: 587}, 'SV': {300000: 585, 300001: 50, 300002: 51, 300003: 51, 300004: 39, 300005: 66, 300006: 418, 300007: 251, 300008: 104}, 'GM': {300000: 40, 300001: 19, 300002: 5, 300003: 4, 300004: 1, 300005: 2, 300006: 5, 300007: 8, 300008: 4}, 'GH': {300000: 361, 300001: 41, 300002: 36, 300003: 53, 300004: 40, 300005: 35, 300006: 162, 300007: 160, 300008: 47}, 'OM': {300000: 566, 300001: 48, 300002: 60, 300003: 59, 300004: 68, 300005: 73, 300006: 381, 300007: 298, 300008: 124}, 'TN': {300000: 306, 300001: 38, 300002: 26, 300003: 45, 300004: 45, 300005: 19, 300006: 243, 300007: 223, 300008: 44}, 'JO': {300000: 557, 300001: 30, 300002: 55, 300003: 63, 300004: 78, 300005: 70, 300006: 415, 300007: 271, 300008: 157}, 'HR': {300000: 600, 300001: 45, 300002: 89, 300003: 92, 300004: 125, 300005: 95, 300006: 569, 300007: 340, 300008: 168}, 'HU': {300000: 600, 300001: 37, 300002: 137, 300003: 129, 300004: 225, 300005: 175, 300006: 600, 300007: 483, 300008: 325}, 'HK': {300000: 600, 300001: 37, 300002: 149, 300003: 186, 300004: 267, 300005: 215, 300006: 600, 300007: 494, 300008: 409}, 'HN': {300000: 591, 300001: 48, 300002: 55, 300003: 55, 300004: 39, 300005: 54, 300006: 421, 300007: 266, 300008: 113}, 'PW': {300000: 11, 300001: 16, 300002: 3, 300003: 0, 300004: 0, 300005: 0, 300006: 1, 300007: 2, 300008: 2}, 'PT': {300000: 600, 300001: 47, 300002: 149, 300003: 142, 300004: 215, 300005: 160, 300006: 600, 300007: 430, 300008: 319}, 'PY': {300000: 600, 300001: 51, 300002: 61, 300003: 52, 300004: 58, 300005: 52, 300006: 441, 300007: 277, 300008: 103}, 'PA': {300000: 600, 300001: 45, 300002: 92, 300003: 77, 300004: 101, 300005: 105, 300006: 553, 300007: 345, 300008: 172}, 'PG': {300000: 32, 300001: 28, 300002: 4, 300003: 6, 300004: 1, 300005: 1, 300006: 6, 300007: 7, 300008: 1}, 'PE': {300000: 600, 300001: 48, 300002: 98, 300003: 104, 300004: 150, 300005: 116, 300006: 600, 300007: 378, 300008: 294}, 'PK': {300000: 594, 300001: 37, 300002: 78, 300003: 99, 300004: 112, 300005: 88, 300006: 419, 300007: 302, 300008: 193}, 'PH': {300000: 600, 300001: 46, 300002: 126, 300003: 142, 300004: 203, 300005: 171, 300006: 600, 300007: 417, 300008: 322}, 'PL': {300000: 600, 300001: 36, 300002: 157, 300003: 158, 300004: 289, 300005: 186, 300006: 600, 300007: 510, 300008: 346}, 'EE': {300000: 600, 300001: 45, 300002: 69, 300003: 84, 300004: 102, 300005: 99, 300006: 569, 300007: 343, 300008: 149}, 'EG': {300000: 600, 300001: 35, 300002: 77, 300003: 96, 300004: 126, 300005: 98, 300006: 526, 300007: 337, 300008: 198}, 'ZA': {300000: 600, 300001: 40, 300002: 168, 300003: 184, 300004: 301, 300005: 260, 300006: 600, 300007: 549, 300008: 366}, 'EC': {300000: 600, 300001: 47, 300002: 90, 300003: 110, 300004: 119, 300005: 128, 300006: 572, 300007: 333, 300008: 202}, 'IT': {300000: 600, 300001: 38, 300002: 253, 300003: 230, 300004: 353, 300005: 290, 300006: 600, 300007: 600, 300008: 461}, 'VN': {300000: 600, 300001: 38, 300002: 107, 300003: 151, 300004: 206, 300005: 128, 300006: 580, 300007: 376, 300008: 304}, 'SB': {300000: 8, 300001: 18, 300002: 2, 300003: 2, 300004: 0, 300005: 0, 300006: 0, 300007: 2, 300008: 1}, 'ZW': {300000: 277, 300001: 28, 300002: 33, 300003: 35, 300004: 18, 300005: 27, 300006: 232, 300007: 108, 300008: 37}, 'SA': {300000: 600, 300001: 37, 300002: 162, 300003: 217, 300004: 264, 300005: 220, 300006: 600, 300007: 526, 300008: 377}, 'ES': {300000: 600, 300001: 37, 300002: 219, 300003: 231, 300004: 323, 300005: 292, 300006: 600, 300007: 588, 300008: 467}, 'MD': {300000: 527, 300001: 41, 300002: 34, 300003: 54, 300004: 43, 300005: 41, 300006: 363, 300007: 259, 300008: 67}, 'MG': {300000: 216, 300001: 24, 300002: 16, 300003: 22, 300004: 5, 300005: 7, 300006: 83, 300007: 41, 300008: 12}, 'UY': {300000: 600, 300001: 38, 300002: 68, 300003: 74, 300004: 92, 300005: 114, 300006: 529, 300007: 329, 300008: 147}, 'UZ': {300000: 278, 300001: 30, 300002: 18, 300003: 35, 300004: 13, 300005: 16, 300006: 240, 300007: 134, 300008: 32}, 'ML': {300000: 255, 300001: 33, 300002: 23, 300003: 24, 300004: 8, 300005: 9, 300006: 110, 300007: 74, 300008: 18}, 'MO': {300000: 577, 300001: 34, 300002: 50, 300003: 56, 300004: 48, 300005: 66, 300006: 388, 300007: 236, 300008: 113}, 'MN': {300000: 303, 300001: 44, 300002: 34, 300003: 21, 300004: 19, 300005: 25, 300006: 243, 300007: 80, 300008: 57}, 'MK': {300000: 394, 300001: 43, 300002: 39, 300003: 37, 300004: 29, 300005: 31, 300006: 285, 300007: 236, 300008: 44}, 'MU': {300000: 501, 300001: 39, 300002: 37, 300003: 56, 300004: 48, 300005: 40, 300006: 328, 300007: 271, 300008: 72}, 'MT': {300000: 600, 300001: 41, 300002: 46, 300003: 61, 300004: 48, 300005: 57, 300006: 451, 300007: 299, 300008: 108}, 'MW': {300000: 135, 300001: 23, 300002: 6, 300003: 10, 300004: 0, 300005: 3, 300006: 46, 300007: 31, 300008: 10}, 'MS': {300000: 16, 300001: 17, 300002: 1, 300003: 6, 300004: 0, 300005: 0, 300006: 0, 300007: 3, 300008: 1}, 'MR': {300000: 85, 300001: 25, 300002: 2, 300003: 17, 300004: 2, 300005: 1, 300006: 7, 300007: 20, 300008: 6}, 'UG': {300000: 260, 300001: 33, 300002: 31, 300003: 42, 300004: 20, 300005: 21, 300006: 111, 300007: 124, 300008: 43}, 'MY': {300000: 600, 300001: 34, 300002: 128, 300003: 138, 300004: 228, 300005: 142, 300006: 596, 300007: 379, 300008: 305}, 'MX': {300000: 600, 300001: 50, 300002: 217, 300003: 224, 300004: 327, 300005: 315, 300006: 600, 300007: 582, 300008: 490}, 'IL': {300000: 600, 300001: 39, 300002: 189, 300003: 223, 300004: 289, 300005: 240, 300006: 600, 300007: 533, 300008: 347}, 'FR': {300000: 600, 300001: 42, 300002: 257, 300003: 234, 300004: 384, 300005: 313, 300006: 600, 300007: 600, 300008: 492}, 'FI': {300000: 600, 300001: 37, 300002: 168, 300003: 199, 300004: 290, 300005: 203, 300006: 600, 300007: 530, 300008: 338}, 'FJ': {300000: 211, 300001: 32, 300002: 19, 300003: 18, 300004: 3, 300005: 6, 300006: 68, 300007: 44, 300008: 10}, 'FM': {300000: 27, 300001: 16, 300002: 1, 300003: 2, 300004: 0, 300005: 4, 300006: 11, 300007: 4, 300008: 0}, 'NI': {300000: 470, 300001: 50, 300002: 36, 300003: 43, 300004: 28, 300005: 31, 300006: 342, 300007: 191, 300008: 70}, 'NL': {300000: 600, 300001: 30, 300002: 216, 300003: 229, 300004: 381, 300005: 286, 300006: 600, 300007: 600, 300008: 447}, 'NO': {300000: 600, 300001: 35, 300002: 229, 300003: 237, 300004: 373, 300005: 300, 300006: 600, 300007: 600, 300008: 546}, 'NA': {300000: 400, 300001: 41, 300002: 35, 300003: 35, 300004: 22, 300005: 43, 300006: 316, 300007: 116, 300008: 52}, 'NE': {300000: 56, 300001: 21, 300002: 4, 300003: 11, 300004: 1, 300005: 2, 300006: 3, 300007: 8, 300008: 3}, 'NG': {300000: 469, 300001: 45, 300002: 38, 300003: 48, 300004: 42, 300005: 42, 300006: 305, 300007: 232, 300008: 77}, 'NZ': {300000: 600, 300001: 44, 300002: 195, 300003: 217, 300004: 305, 300005: 269, 300006: 600, 300007: 552, 300008: 397}, 'NP': {300000: 207, 300001: 15, 300002: 20, 300003: 20, 300004: 6, 300005: 11, 300006: 103, 300007: 68, 300008: 12}, 'CH': {300000: 600, 300001: 30, 300002: 223, 300003: 235, 300004: 376, 300005: 310, 300006: 600, 300007: 600, 300008: 479}, 'CO': {300000: 600, 300001: 41, 300002: 122, 300003: 122, 300004: 223, 300005: 160, 300006: 600, 300007: 404, 300008: 337}, 'CN': {300000: 551, 300001: 24, 300002: 141, 300003: 169, 300004: 204, 300005: 97, 300006: 368, 300007: 258, 300008: 233}, 'CL': {300000: 600, 300001: 48, 300002: 153, 300003: 156, 300004: 275, 300005: 182, 300006: 600, 300007: 469, 300008: 390}, 'CA': {300000: 600, 300001: 38, 300002: 280, 300003: 250, 300004: 417, 300005: 421, 300006: 600, 300007: 600, 300008: 600}, 'CG': {300000: 237, 300001: 32, 300002: 29, 300003: 48, 300004: 43, 300005: 37, 300006: 114, 300007: 134, 300008: 53}, 'CZ': {300000: 600, 300001: 43, 300002: 154, 300003: 153, 300004: 290, 300005: 191, 300006: 600, 300007: 502, 300008: 376}, 'CY': {300000: 594, 300001: 46, 300002: 60, 300003: 66, 300004: 38, 300005: 62, 300006: 418, 300007: 304, 300008: 88}, 'CR': {300000: 600, 300001: 50, 300002: 94, 300003: 108, 300004: 150, 300005: 140, 300006: 582, 300007: 363, 300008: 233}, 'CV': {300000: 124, 300001: 27, 300002: 7, 300003: 8, 300004: 4, 300005: 4, 300006: 30, 300007: 35, 300008: 6}, 'SZ': {300000: 126, 300001: 21, 300002: 6, 300003: 16, 300004: 1, 300005: 4, 300006: 45, 300007: 33, 300008: 7}, 'KG': {300000: 237, 300001: 34, 300002: 7, 300003: 28, 300004: 10, 300005: 9, 300006: 159, 300007: 105, 300008: 11}, 'KE': {300000: 536, 300001: 47, 300002: 61, 300003: 71, 300004: 69, 300005: 70, 300006: 359, 300007: 276, 300008: 118}, 'SR': {300000: 97, 300001: 23, 300002: 8, 300003: 11, 300004: 4, 300005: 1, 300006: 24, 300007: 20, 300008: 7}, 'KH': {300000: 407, 300001: 46, 300002: 41, 300003: 43, 300004: 32, 300005: 43, 300006: 312, 300007: 188, 300008: 72}, 'KN': {300000: 144, 300001: 31, 300002: 19, 300003: 10, 300004: 3, 300005: 1, 300006: 33, 300007: 35, 300008: 11}, 'ST': {300000: 5, 300001: 16, 300002: 0, 300003: 1, 300004: 0, 300005: 0, 300006: 0, 300007: 2, 300008: 0}, 'SK': {300000: 600, 300001: 41, 300002: 104, 300003: 104, 300004: 181, 300005: 159, 300006: 600, 300007: 430, 300008: 258}, 'KR': {300000: 346, 300001: 37, 300002: 28, 300003: 71, 300004: 41, 300005: 40, 300006: 140, 300007: 118, 300008: 53}, 'SI': {300000: 600, 300001: 45, 300002: 67, 300003: 82, 300004: 94, 300005: 79, 300006: 548, 300007: 336, 300008: 137}, 'KW': {300000: 600, 300001: 33, 300002: 100, 300003: 131, 300004: 212, 300005: 141, 300006: 600, 300007: 430, 300008: 301}, 'SN': {300000: 330, 300001: 32, 300002: 33, 300003: 38, 300004: 18, 300005: 27, 300006: 220, 300007: 145, 300008: 31}, 'SL': {300000: 170, 300001: 22, 300002: 8, 300003: 12, 300004: 2, 300005: 3, 300006: 68, 300007: 28, 300008: 9}, 'SC': {300000: 153, 300001: 34, 300002: 12, 300003: 9, 300004: 3, 300005: 14, 300006: 31, 300007: 30, 300008: 16}, 'KZ': {300000: 600, 300001: 31, 300002: 62, 300003: 64, 300004: 76, 300005: 75, 300006: 423, 300007: 309, 300008: 127}, 'KY': {300000: 323, 300001: 39, 300002: 37, 300003: 20, 300004: 12, 300005: 12, 300006: 145, 300007: 99, 300008: 25}, 'SG': {300000: 600, 300001: 44, 300002: 160, 300003: 178, 300004: 292, 300005: 194, 300006: 600, 300007: 486, 300008: 352}, 'SE': {300000: 600, 300001: 31, 300002: 248, 300003: 244, 300004: 380, 300005: 332, 300006: 600, 300007: 600, 300008: 538}, 'DO': {300000: 600, 300001: 43, 300002: 84, 300003: 83, 300004: 98, 300005: 85, 300006: 508, 300007: 327, 300008: 180}, 'DM': {300000: 95, 300001: 26, 300002: 7, 300003: 10, 300004: 0, 300005: 3, 300006: 33, 300007: 20, 300008: 3}, 'DK': {300000: 600, 300001: 22, 300002: 207, 300003: 241, 300004: 362, 300005: 291, 300006: 600, 300007: 600, 300008: 427}, 'DE': {300000: 600, 300001: 32, 300002: 269, 300003: 242, 300004: 407, 300005: 353, 300006: 600, 300007: 600, 300008: 531}, 'YE': {300000: 219, 300001: 24, 300002: 11, 300003: 20, 300004: 11, 300005: 6, 300006: 205, 300007: 89, 300008: 30}, 'DZ': {300000: 292, 300001: 30, 300002: 26, 300003: 52, 300004: 21, 300005: 22, 300006: 226, 300007: 206, 300008: 63}, 'US': {300000: 600, 300001: 44, 300002: 323, 300003: 264, 300004: 482, 300005: 485, 300006: 600, 300007: 600, 300008: 600}, 'LB': {300000: 600, 300001: 33, 300002: 74, 300003: 89, 300004: 115, 300005: 102, 300006: 533, 300007: 315, 300008: 193}, 'LC': {300000: 175, 300001: 29, 300002: 16, 300003: 13, 300004: 3, 300005: 9, 300006: 51, 300007: 39, 300008: 6}, 'LA': {300000: 246, 300001: 34, 300002: 8, 300003: 11, 300004: 7, 300005: 10, 300006: 103, 300007: 33, 300008: 17}, 'TW': {300000: 600, 300001: 40, 300002: 128, 300003: 169, 300004: 267, 300005: 211, 300006: 600, 300007: 447, 300008: 389}, 'TT': {300000: 427, 300001: 36, 300002: 39, 300003: 47, 300004: 25, 300005: 35, 300006: 287, 300007: 177, 300008: 46}, 'TR': {300000: 600, 300001: 47, 300002: 154, 300003: 201, 300004: 325, 300005: 263, 300006: 600, 300007: 553, 300008: 423}, 'LK': {300000: 428, 300001: 48, 300002: 33, 300003: 45, 300004: 36, 300005: 35, 300006: 295, 300007: 182, 300008: 85}, 'LV': {300000: 600, 300001: 39, 300002: 71, 300003: 89, 300004: 113, 300005: 81, 300006: 581, 300007: 338, 300008: 139}, 'LT': {300000: 600, 300001: 46, 300002: 78, 300003: 85, 300004: 122, 300005: 81, 300006: 584, 300007: 339, 300008: 165}, 'LU': {300000: 600, 300001: 38, 300002: 111, 300003: 128, 300004: 172, 300005: 137, 300006: 600, 300007: 437, 300008: 242}, 'LR': {300000: 143, 300001: 24, 300002: 10, 300003: 19, 300004: 2, 300005: 3, 300006: 30, 300007: 36, 300008: 11}, 'TH': {300000: 600, 300001: 38, 300002: 124, 300003: 158, 300004: 235, 300005: 187, 300006: 600, 300007: 464, 300008: 321}, 'TD': {300000: 87, 300001: 19, 300002: 7, 300003: 9, 300004: 2, 300005: 7, 300006: 20, 300007: 14, 300008: 7}, 'TC': {300000: 248, 300001: 31, 300002: 16, 300003: 36, 300004: 15, 300005: 7, 300006: 151, 300007: 83, 300008: 19}, 'VC': {300000: 109, 300001: 19, 300002: 7, 300003: 6, 300004: 0, 300005: 1, 300006: 29, 300007: 31, 300008: 5}, 'AE': {300000: 600, 300001: 37, 300002: 161, 300003: 190, 300004: 282, 300005: 241, 300006: 600, 300007: 546, 300008: 412}, 'VE': {300000: 266, 300001: 35, 300002: 39, 300003: 64, 300004: 40, 300005: 56, 300006: 235, 300007: 173, 300008: 64}, 'AG': {300000: 181, 300001: 34, 300002: 5, 300003: 13, 300004: 4, 300005: 20, 300006: 62, 300007: 35, 300008: 7}, 'VG': {300000: 182, 300001: 27, 300002: 17, 300003: 10, 300004: 7, 300005: 11, 300006: 45, 300007: 49, 300008: 11}, 'AI': {300000: 88, 300001: 20, 300002: 4, 300003: 7, 300004: 0, 300005: 2, 300006: 29, 300007: 23, 300008: 4}, 'IS': {300000: 600, 300001: 40, 300002: 113, 300003: 118, 300004: 191, 300005: 130, 300006: 545, 300007: 389, 300008: 241}, 'AM': {300000: 409, 300001: 37, 300002: 36, 300003: 50, 300004: 45, 300005: 42, 300006: 322, 300007: 245, 300008: 73}, 'AL': {300000: 325, 300001: 54, 300002: 19, 300003: 34, 300004: 23, 300005: 22, 300006: 249, 300007: 198, 300008: 49}, 'AO': {300000: 302, 300001: 58, 300002: 36, 300003: 37, 300004: 20, 300005: 23, 300006: 252, 300007: 160, 300008: 38}, 'AR': {300000: 600, 300001: 51, 300002: 149, 300003: 175, 300004: 256, 300005: 193, 300006: 600, 300007: 451, 300008: 342}, 'AU': {300000: 600, 300001: 37, 300002: 285, 300003: 241, 300004: 404, 300005: 390, 300006: 600, 300007: 600, 300008: 573}, 'AT': {300000: 600, 300001: 28, 300002: 163, 300003: 187, 300004: 332, 300005: 221, 300006: 600, 300007: 562, 300008: 354}, 'IN': {300000: 600, 300001: 46, 300002: 163, 300003: 225, 300004: 286, 300005: 257, 300006: 600, 300007: 532, 300008: 396}, 'TZ': {300000: 266, 300001: 44, 300002: 25, 300003: 31, 300004: 13, 300005: 17, 300006: 109, 300007: 115, 300008: 27}, 'AZ': {300000: 470, 300001: 37, 300002: 34, 300003: 53, 300004: 33, 300005: 46, 300006: 351, 300007: 246, 300008: 75}, 'IE': {300000: 600, 300001: 37, 300002: 206, 300003: 200, 300004: 270, 300005: 210, 300006: 600, 300007: 514, 300008: 355}, 'ID': {300000: 600, 300001: 45, 300002: 112, 300003: 122, 300004: 201, 300005: 150, 300006: 600, 300007: 429, 300008: 320}, 'UA': {300000: 600, 300001: 37, 300002: 98, 300003: 113, 300004: 224, 300005: 146, 300006: 600, 300007: 397, 300008: 273}, 'QA': {300000: 600, 300001: 39, 300002: 78, 300003: 104, 300004: 108, 300005: 93, 300006: 512, 300007: 337, 300008: 217}, 'MZ': {300000: 297, 300001: 41, 300002: 30, 300003: 28, 300004: 9, 300005: 11, 300006: 192, 300007: 123, 300008: 30}}


df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/app-tech.store.app-rank.v1/fact/date={}/device_code={}/".format("2019-07-29", "tv-os-tv"))
df_unified = df.groupBy(["country_code","category_id"]).agg(F.count("free_download").alias("free_download_count"),F.count("paid_download").alias("paid_download_count"), F.count("revenue").alias("revenue_count")).collect()

unified_result = {}
for data in df_unified:
    unified_count = sum([data.free_download_count, data.paid_download_count, data.revenue_count])
    diff_count = raw_result[data.country_code][data.category_id]-unified_count
    if diff_count>0:
        print "{} {} {}".format(data.country_code,data.category_id, diff_count)
    



In [0]:


import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list

begin_date = datetime.datetime.strptime("2015-12-23", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2019-11-26", '%Y-%m-%d')
date_list = get_date_list(begin_date, end_date, "D")

print "%table date\tr_count\tu_count\tdiff"


date_list = ["2018-02-21", "2019-07-29"]

for date_str in date_list:
    try:
        u_count = get_unified_count_by_date(date_str)
        r_count = TvOsTvRaw().get_raw_data_by_date(date_str)
        print "{}\t{}\t{}\t{}".format(date_str, r_count, u_count, r_count-u_count)
    except Exception as e:
        print "{}\t{}\t{}\t{}".format(date_str, 0, 0, -100)
        
    

In [0]:


df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-free/unified/app-tech.store.app-rank.v1/fact/date={}/device_code={}/".format("2019-07-29", "tv-os-tv"))
df_unified = df.filter("category_id=300001 and country_code='PT'").orderBy("free_download").show(1000)



In [0]:

{'date': '2019-07-29', 'country': 'MR', 'category': 360, 'rank': ['1-927559995', '2-1117365978', '3-1086978383', '4-435890283', '5-533412119', '6-1104945393', '7-613398383', '8-689165391', '9-1245370788', '10-969247886', '11-969247886', '12-428920239', '13-561572290', '14-836558771', '15-719219382', '16-668416040', '17-1138436619', '18-1146875428', '19-428267291', '20-437760529', '21-1280277415', '22-1113619083'], 'feed': '0'}
{'date': '2019-07-29', 'country': 'MR', 'category': 360, 'rank': ['1-1042299727'], 'feed': '1'}
{'date': '2019-07-29', 'country': 'MR', 'category': 360, 'rank': ['1-533412119', '2-1042299727'], 'feed': '2'}
{'date': '2019-07-29', 'country': 'PT', 'category': 360, 'rank': ['1-836558771', '2-927559995', '3-428267291', '4-613398383', '5-689165391', '6-1292181007', '7-1042608915', '8-1113619083', '9-1104945393', '10-1280277415', '11-428372830', '12-1086978383', '13-969247886', '14-719219382', '15-1095795448', '16-1095795448', '17-668416040'], 'feed': '0'}
{'date': '2019-07-29', 'country': 'PT', 'category': 360, 'rank': ['1-307459878', '2-291322250', '3-936966605', '4-931936670', '5-907759227', '6-632759992', '7-1042299727', '8-1120547622', '9-794730213', '10-1343246862'], 'feed': '1'}
{'date': '2019-07-29', 'country': 'PT', 'category': 360, 'rank': ['1-927559995', '2-428920239', '3-428267291', '4-689165391', '5-613398383', '6-1138436619', '7-1104945393', '8-426811787', '9-430537586', '10-1058370532', '11-533412119', '12-307459878', '13-1042608915', '14-719219382', '15-428372830', '16-291322250', '17-1113619083', '18-931936670', '19-936966605', '20-435890283'], 'feed': '2'}



In [0]:


{'date': '2019-10-05', 'country': '143547', 'category': 200037, 'rank': ['520258144', '747824928'], 'feed': '200'}
{'date': '2019-10-05', 'country': '143547', 'category': 200037, 'rank': [''], 'feed': '201'}
{'date': '2019-10-05', 'country': '143547', 'category': 200037, 'rank': [''], 'feed': '202'}
